<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sharedmobility.ch - Validation of GBFS-Feeds

In [1]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 11.6 MB/s eta 0:00:00


In [2]:
import requests
import pandas as pd
from datetime import datetime
import json

In [3]:
# GBFS Feeds abfragen
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'
feeds = requests.get(url).json()

In [4]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
providers = list(gbfsfeeds['provider'])

In [5]:
# Nur einen Provider temporär einstellen!
providers = ['bolt_zurich','velospot','carvelo2go','bird-biel']
providers

['bolt_zurich', 'velospot', 'carvelo2go', 'bird-biel']

In [6]:
# Validieren!
JsonResponse = []
providerList2 = []
HasErrors = []
NumberOfErrors = []

for provider in providers:
    try:
      headers = {
          'accept': '*/*',
          'Content-Type': 'application/json',
          'User-Agent': 'geoinformation@bfe.admin.ch',
      }

      json_data = {
          'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ provider +'/gbfs',
          'options': {
              'freefloating': False,
              'docked': False,
              'version': None,
              'auth': {
                  'type': None,
                  'basicAuth': {
                      'user': None,
                      'password': None,
                  },
                  'bearerToken': {
                      'token': None,
                  },
                  'oauthClientCredentialsGrant': {
                      'user': None,
                      'password': None,
                      'tokenUrl': None,
                  },
              },
          },
      }
      response = requests.post('https://gbfs-validator.netlify.app/.netlify/functions/validator', headers=headers, json=json_data).json()

      providerList2.append(provider)
      JsonResponse.append(response)
      HasErrors.append(response['summary']['hasErrors'])
      NumberOfErrors.append(response['summary']['errorsCount'])

    except:
      providerList2.append(provider)
      JsonResponse.append(response)
      HasErrors.append('nan')
      NumberOfErrors.append('nan')

In [7]:
ValidationResults = pd.json_normalize(JsonResponse)

In [8]:
ValidationResults['provider'] = providerList2

In [11]:
ValidationResults

,files,summary.validatorVersion,summary.version.detected,summary.version.validated,summary.hasErrors,summary.errorsCount,provider,Validator
0,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,bolt_zurich,https://gbfs-validator.netlify.app/validator?u...
1,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,True,13,velospot,https://gbfs-validator.netlify.app/validator?u...
2,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,carvelo2go,https://gbfs-validator.netlify.app/validator?u...
3,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,bird-biel,https://gbfs-validator.netlify.app/validator?u...


In [9]:
ValidationResults['Validator'] = 'https://gbfs-validator.netlify.app/validator?url=https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ ValidationResults['provider']+'/gbfs'

In [12]:
ValidationResults = ValidationResults.drop(columns=['files','summary.validatorVersion','summary.version.detected'])

In [40]:
ValidationResults

,summary.version.validated,summary.hasErrors,summary.errorsCount,provider,Validator,newCol
0,2.3,False,0,bolt_zurich,https://gbfs-validator.netlify.app/validator?u...,schwarz
1,2.3,True,13,velospot,https://gbfs-validator.netlify.app/validator?u...,grün
2,2.3,False,0,carvelo2go,https://gbfs-validator.netlify.app/validator?u...,schwarz
3,2.3,False,0,bird-biel,https://gbfs-validator.netlify.app/validator?u...,schwarz


In [ ]:
ValidationResults.to_csv("GBFS_validated.csv",header=True,index=False)

# Create Statusbadges für Übersicht auf Github

In [75]:
# Funktion die den Farbcode zuweist.
def CreateBadgeColour(row):
  if row['summary.hasErrors']:
    return "red"
  else:
    return "green"

In [76]:
ValidationResults['Colour']= ValidationResults.apply(lambda row: CreateBadgeColour(row), axis=1)

In [73]:
ValidationResults['Badge']= '- [![Dokumentation](https://badgen.net/badge/' + ValidationResults['provider'] + '/valid/' + ValidationResults['Colour'] + '?icon=github)](' + ValidationResults['Validator'] + ')'
ValidationResults['Badge']


0    - [![Dokumentation](https://badgen.net/badge/b...
1    - [![Dokumentation](https://badgen.net/badge/v...
2    - [![Dokumentation](https://badgen.net/badge/c...
3    - [![Dokumentation](https://badgen.net/badge/b...
Name: Badge, dtype: object